# 1. Get VizDoom Up and Running

In [ ]:
!pip install vizdoom

In [ ]:
!cd github & git clone https://github.com/Farama-Foundation/ViZDoom.git

In [1]:
# Import vizdoom for game environment
from vizdoom import *
# Import random to take random actions
import random
# Import time to slow down game, sleep between frames
import time
# Import numpy for identity matrix
import numpy as np

In [ ]:
# Setup Game
game = DoomGame()
game.load_config('github/ViZDoom/scenarios/deadly_corridor_s1.cfg')
game.init()

In [ ]:
# Define the actions we can take in the environment - Move left, move right, attack
actions = np.identity(7, dtype=np.uint8) 

In [ ]:
state = game.get_state()

In [ ]:
state.game_variables

In [ ]:
random.choice(actions)

In [ ]:
# Loop through episodes
episodes = 10 # Number of games to play
for episode in range(episodes):
    # Create new episode or game
    game.new_episode()
    # Check that the game isn't done
    while not game.is_episode_finished():
        # Get the game state
        state = game.get_state()
        # Get the game image
        img = state.screen_buffer
        # Get the game variables (in this case, ammo)
        info = state.game_variables
        # Take an action. Pass in frame skip to give AI time to process each action reward
        reward = game.make_action(random.choice(actions), 4)
        # Print the reward for each action
        # print('Reward:', reward)
        time.sleep(0.02)
    # Print total reward for full game
    print('Result:', game.get_total_reward())
    time.sleep(2)

In [ ]:
game.close()

# 2. Converting the Environment to a Gym Envrionment

In [ ]:
!pip install gym

In [2]:
# Import environment base class from OpenAI Gym
from gym import Env
# Import gym spaces
from gym.spaces import Discrete, Box
# Import opencv, Used to greyscale observations to make processing environment faster
import cv2

In [3]:
# Create VIZDoom OpenAI Gym Environment
class VizDoomGym(Env):
    
    # Function that is called when we start the environment
    def __init__(self, render=False, config='github/ViZDoom/scenarios/deadly_corridor_s1.cfg'):
        # Inherit from Env
        super().__init__()
        
        # Setup the game
        self.game = DoomGame()
        self.game.load_config(config) # Pass in whatever environment you need from the scenarios folder.
        
        # Game Variables:
        self.damage_taken = 0
        self.hitcount = 0
        self.ammo = 52
        
        # Define whether or not to render the game window.
        # Rendering the window takes away from computing power, so disabling is ideal for testing
        if render == False:
            self.game.set_window_visible(False)
        else:
            self.game.set_window_visible(True)
        
        self.game.init()
        
        # Create the action and observation space
        self.observation_space = Box(low=0, high=255, shape=(100, 160, 1), dtype=np.uint8)
        self.action_space = Discrete(7)
    
    # How we take a step in the environment
    def step(self, action):
        # Specify action and take step within game
        actions = np.identity(7, dtype=np.uint8)
        movement_reward = self.game.make_action(actions[action], 4)
        
        reward = 0
        # Get all other stuff we need to return
        if self.game.get_state():
            state = self.game.get_state().screen_buffer
            state = self.greyscale(state)
            
            # Reward shaping
            game_variables = self.game.get_state().game_variables
            health, damage_taken, hitcount, ammo = game_variables
            
            # Calculate reward deltas
            damage_taken_delta = -damage_taken + self.damage_taken
            self.damage_taken = damage_taken
            hitcount_delta = hitcount - self.hitcount
            self.hitcount = hitcount
            ammo_delta = ammo - self.ammo
            self.ammo = ammo
            
            reward = movement_reward + damage_taken_delta*10 + hitcount_delta*200 + ammo_delta*5
            info = ammo
        else:
            state = np.zeros(self.observation_space.shape)
            info = 0
            
        info = {"info": info}
            
        done = self.game.is_episode_finished()
        
        return state, reward, done, info
    
    # Define how to render the game or environment. ViZDOom already defines this for us, so just pass.
    def render():
        pass
    
    # What happens when we start a new gmae
    def reset(self):
        self.game.new_episode()
        state = self.game.get_state().screen_buffer
        return self.greyscale(state)
    
    # Custom function. Greyscale the game frame and resize it
    def greyscale(self, observation):
        # Reshape the observation array for cvtColor and change color channels
        grey = cv2.cvtColor(np.moveaxis(observation, 0, -1), cv2.COLOR_BGR2GRAY)
        # Resize the image and scale down so there are less pixels to process
        resize = cv2.resize(grey, (160, 100), interpolation=cv2.INTER_CUBIC)
        state = np.reshape(resize, (100, 160, 1))
        return state
    
    # Call to close down the game
    def close(self):
        self.game.close()

In [ ]:
env = VizDoomGym(render=True)

In [ ]:
env.reset()

In [ ]:
env.close()

In [ ]:
# Import Environment checker
from stable_baselines3.common import env_checker

In [ ]:
env_checker.check_env(env)

# 3. View State

In [ ]:
!pip install matplotlib

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.imshow(cv2.cvtColor(state, cv2.COLOR_BGR2RGB))

# 4. Log the Results

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

In [ ]:
!pip install stable-baselines3[extra]

In [4]:
# Import os for file navigation
import os
# Import callback class from stable baselines 3
from stable_baselines3.common.callbacks import BaseCallback

In [5]:
class TrainAndLoggingCallback(BaseCallback):
    
    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path
        
    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)
    
    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)
            
        return True

In [6]:
CHECKPOINT_DIR = './train/train_deadly_corridor'
LOG_DIR = './logs/log_deadly_corridor'

In [7]:
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

# 5. Train Model using Curriculum

In [8]:
# Import the PPO algorithm for training
from stable_baselines3 import PPO

In [9]:
env = VizDoomGym(config='github/ViZDoom/scenarios/deadly_corridor_s2.cfg')

In [14]:
# CnnPolicy because we are passing in an image
# Cnn = Convolution neural network

# model = PPO('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=1, learning_rate=0.0001, n_steps=4096)
# Change hyperparameters for more stability in model training
model = PPO('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=1, learning_rate=0.00001, n_steps=8192, clip_range=.1, gamma=0.95, gae_lambda=.9)
# model = PPO.load('./train/train_deadly_corridor/best_model_560000')

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


In [15]:
model.learn(total_timesteps=80000, callback=callback)

Logging to ./logs/log_deadly_corridor\PPO_13
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 91.7     |
|    ep_rew_mean     | 55.2     |
| time/              |          |
|    fps             | 56       |
|    iterations      | 1        |
|    time_elapsed    | 143      |
|    total_timesteps | 8192     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 77.5         |
|    ep_rew_mean          | 58           |
| time/                   |              |
|    fps                  | 58           |
|    iterations           | 2            |
|    time_elapsed         | 279          |
|    total_timesteps      | 16384        |
| train/                  |              |
|    approx_kl            | 0.0024412374 |
|    clip_fraction        | 0.143        |
|    clip_range           | 0.1          |
|    entropy_loss         | -1.94        |
|    explaine

KeyboardInterrupt: 

In [16]:
env.close()

In [ ]:
env = VizDoomGym(config='github/ViZDoom/scenarios/deadly_corridor_s2.cfg')
model.set_env(env)
model = PPO.load('./train/train_deadly_corridor/best_model_340000')
model.learn(total_timesteps=40000, callback=callback)

In [ ]:
env = VizDoomGym(config='github/ViZDoom/scenarios/deadly_corridor_s3.cfg')
model.set_env(env)
model.learn(total_timesteps=100000, callback=callback)

In [ ]:
env = VizDoomGym(config='github/ViZDoom/scenarios/deadly_corridor_s4.cfg')
model.set_env(env)
model.learn(total_timesteps=40000, callback=callback)

In [ ]:
env = VizDoomGym(config='github/ViZDoom/scenarios/deadly_corridor_s5.cfg')
model.set_env(env)
model.learn(total_timesteps=40000, callback=callback)

To view the logs run `tensorboard --logdir=.` in the PPO dir of the model run.  
  
Explaining the data:  
> 1. `ep_len_mean`: Mean episode length (averaged over stats_window_size episodes, 100 by default)
> 2. `ep_rew_mean`: Mean episodic training reward (averaged over stats_window_size episodes, 100 by default).
> 3. `approx_kl`: approximate mean KL divergence between old and new policy (for PPO), it is an estimation of how much changes happened in the update
> 4. `clip_fraction`: mean fraction of surrogate loss that was clipped (above clip_range threshold) for PPO.
> 5. `clip_range`: Current value of the clipping factor for the surrogate loss of PPO
> 6. `critic_loss`: Current value for the critic function loss for off-policy algorithms, usually error between value function output and TD(0), temporal difference estimate
> 7. `learning_rate`: Current learning rate value
> 8. `loss`: Current total loss value
> 9. `n_updates`: Number of gradient updates applied so far
> 14. `policy_gradient_loss`: Current value of the policy gradient loss (its value does not have much meaning)
> 15. `value_loss`: Current value for the value function loss for on-policy algorithms

# 6. Test the Model

In [ ]:
# Import eval policy to test agent
from stable_baselines3.common.evaluation import evaluate_policy

In [ ]:
# Reload model from disc
# model = PPO.load('./train/train_deadly_corridor/best_model_560000_nick')

In [ ]:
# Create rendered environment
env = VizDoomGym(render=True, config="github/ViZDoom/scenarios/deadly_corridor_s3.cfg")

In [ ]:
# Evaluate mean reward for 100 games
mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=100)

In [ ]:
mean_reward

In [ ]:
for episode in range(10):
    obs = env.reset()
    done = False
    total_reward = 0
    while not done:
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        # time.sleep(0.125)
        total_reward += reward
    print('Total reward for episode {} is {}'.format(episode, total_reward))
    time.sleep(2)

In [ ]:
env.close()